In [4]:
from helper_functions import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.tokenize import sent_tokenize
import torch
from transformers import BertTokenizer, BertModel

seed=25 #for random state / reproducibility
pd.set_option('display.width', 200)
pd.set_option('display.max_colwidth', 200) 

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
%pip install transformers sentence-transformers torch nltk
%pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


In [6]:
df = pd.read_csv("ai_ml_papers.csv")

C:\Users\ekabu\AppData\Local\Temp\ipykernel_47184\149443613.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("ai_ml_papers.csv")


Method 1. BERT Embeddings with Text Rank Summary

In [12]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download("punkt")
# Load a pre-trained BERT model for sentence embeddings
model_BERT = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

def summarize_BERT_TextRank(text):
    text = preprocess(text) #preprocess
    sentences = sent_tokenize(text)

    # Convert each sentence into an embedding
    sentence_embeddings = model_BERT.encode(sentences, convert_to_tensor=True)

    # Compute the mean embedding (representing the overall document)
    mean_embedding = torch.mean(sentence_embeddings, dim=0)

    # Compute cosine similarity of each sentence to the overall document embedding
    cos_similarities = cosine_similarity(sentence_embeddings.cpu().numpy(), mean_embedding.cpu().numpy().reshape(1, -1))

    # Rank sentences by similarity
    top_n = 5  # Select the top 5 most representative sentences
    top_sentence_indices = np.argsort(cos_similarities, axis=0)[-top_n:].flatten()

    # Sort indices to maintain the original order of sentences
    top_sentence_indices = sorted(top_sentence_indices)

    # Extract the summary
    summary = " ".join([sentences[i] for i in top_sentence_indices])

    return summary
  

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ekabu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Method 2. Using google/pegasus-xsum model

In [13]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

# Load the model and tokenizer
model_name = "google/pegasus-arxiv"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-arxiv and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
def summarize_pegasus(text):
    text = preprocess(text) #preprocess
    sentences = sent_tokenize(text)
    
    chunks = chunk_text(sentences, tokenizer)
    
    summary_list = []

    for chunk in chunks:
        inputs = tokenizer(chunk, return_tensors="pt", max_length=1024, truncation=True)
        summary_ids = model.generate(inputs["input_ids"], max_length=200, num_beams=5)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summary_list.append(summary)

    # Combine all summarized chunks
    summary = " ".join(summary_list)
    
    return summary


Generate Summary

In [15]:
n_sample = 3

df_sample = df.sample(n=n_sample, random_state=seed)
#extract and append full-text
df_sample['full_text'] = df_sample['id'].apply(extract_pdf_text)
df_sample['summary_bert_textRank'] = df_sample['full_text'].apply(summarize_BERT_TextRank)
df_sample['summary_pegasus_pretrained'] = df_sample['full_text'].apply(summarize_pegasus)

In [16]:

print(df_sample[['id', 'title', 'summary_bert_textRank']])
print(df_sample[['id', 'title', 'summary_pegasus_pretrained']])


                id                                                             title  \
91709   2101.10007      Adaptive Scheduling for Machine Learning Tasks over Networks   
89614   2012.12109  Enhance Convolutional Neural Networks with Noise Incentive Block   
108316  2107.11481          Similarity Based Label Smoothing For Dialogue Generation   

                                                                                                                                                                                          summary_bert_textRank  
91709   paper , whenever expectation iterates wk taken , expectation data collected time k. b. scheduling problem single task given modeling machine learning task needs solved , scheduling problem follows...  
89614   figure 1 demonstrates ﬂatness degradation affects several typical applications : ( ) semantic image synthesis synthesizes arxiv:2012.12109v2 [ cs.cv ] 9 jun 2021 photo-realistic textures piece-wis...  
108316  language 